# Estimate Total Costings

To estimate monhtly costings from:
- Prompt engineering from GPT3.5
- Image generation from DALL-E

https://openai.com/pricing

####  GPT4
| Model | Input | Output |
| -- | -- | -- |
| 8K context | $0.03 / 1K tokens | $0.06 / 1K tokens |
| 32K context | $0.06 / 1K tokens | $0.12 / 1K tokens|

#### GPT3.5-turbo
| Model | Input | Output |
| -- | -- | -- |
|4K context	| $0.0015 / 1K tokens |$0.002 / 1K tokens |
|16K context | $0.003 / 1K tokens | $0.004 / 1K tokens |


#### DALL-E
| Resolution | Price |
| -- | -- |
| 1024×1024 | $0.020 / image |
| 512×512 | $0.018 / image |
| 256×256 | $0.016 / image |

In [46]:
# Check constants from website before running
gpt4_8k = {'name': 'gpt4_8k', 'cost': {'input': 0.03, 'output': 0.06}}
gpt4_32k = {'name': 'gpt4_32k', 'cost': {'input': 0.06, 'output': 0.12}}

gpt35t_4k = {'name': 'gpt35t_4k', 'cost': {'input': 0.0015, 'output': 0.002}}
gpt35t_16k = {'name': 'gpt35t_16k', 'cost': {'input': 0.003, 'output': 0.004}}

dalle_1024 = {'name': 'dalle_1024', 'cost': 0.02}
dalle_512 = {'name': 'dalle_512', 'cost': 0.018}
dalle_256 = {'name': 'dallee_256', 'cost': 0.016}

In [37]:
import os
import boto3
import json
import itertools
import numpy as np
from dotenv import load_dotenv

In [2]:
load_dotenv()

aws_access_key = os.environ['aws_access_key']
aws_secret_key = os.environ['aws_secret_key']
aws_region = os.environ['aws_region']
currents_api_key = os.environ['currents_api_key']

In [5]:
s3 = boto3.client(
        's3',
        aws_access_key_id=aws_access_key,
        aws_secret_access_key=aws_secret_key,
        region_name=aws_region
)

bucket_name = 'glimpse-landing-dev'
objs = s3.list_objects_v2(Bucket=bucket_name)['Contents']

In [6]:
all_feeds = []

for obj in objs:
    response = s3.get_object(Bucket=bucket_name, Key=obj['Key'])
    file_content = response['Body'].read().decode('utf-8')
    json_content = json.loads(file_content)
    all_feeds.append(json_content)

In [8]:
all_news = [r['news'] for r in all_feeds]
all_titles = [[news['title'] for news in day] for day in all_news]
all_words = [' '.join(titles) for titles in all_titles]

#### Input Prompt

In [53]:
prompt = '''

Can you adjust the following sentence surrounded by ``` with the following:
- Adjust the sentence into one describing a person or a scene in less than 25 words
- Replace any names with a description of their age, gender and ethnicity
- Replace any political agendas to something more ambiguous
- Be as descriptive as possible about the subject so that DALL-E can comprehend
- Add in "A street photo of" to the start of the response
- Add in "shot by a Leica." at the back of the response

```Bangladesh's worst ever dengue outbreak a 'canary in the coal mine' for climate crisis, WHO expert warns```
A street photo of a concerned middle-aged Asian woman, her brows furrowed, emphasizing climate impact, shot by a Leica.
'''



In [54]:
input_template_tokens = len(prompt.split(' '))

title_tokens = [len(article.split(' ')) for article in all_titles[0]]
input_avg_input_tokens = np.round(np.mean(title_tokens), 0)

output_max_tokens = 25

total_est_input_tokens = input_template_tokens + input_avg_input_tokens
total_est_output_tokens = output_max_tokens

In [55]:
def total_monthly_cost(
    gpt_model: dict, 
    img_cost: dict, 
    input: float, 
    output: float) -> float:

    days_in_month = 30
    daily_gpt_cost_input = (input * gpt_model['cost']['input']) / 1000
    daily_gpt_cost_output = (output * gpt_model['cost']['output']) / 1000
    daily_img_cost = img_cost['cost']
    total_monthly_cost = (daily_gpt_cost_input + daily_gpt_cost_output + daily_img_cost) * days_in_month

    return np.round(total_monthly_cost, 2) 

In [56]:
prompt_models = [gpt4_8k, gpt4_32k, gpt35t_4k, gpt35t_16k]
img_models = [dalle_256, dalle_512, dalle_1024]
inputs = [total_est_input_tokens]
outputs = [total_est_output_tokens] 

combined = [prompt_models, img_models, inputs, outputs]

combinations = list(itertools.product(*combined))

In [57]:
for k in combinations:
    print(
        f'''
            Prompt Model: {k[0]['name']}
            Image Resolution: {k[1]['name']}
            Total monthly cost: ${total_monthly_cost(k[0], k[1], k[2], k[3])}
          '''
          )


            Prompt Model: gpt4_8k
            Image Resolution: dallee_256
            Total monthly cost: $0.64
          

            Prompt Model: gpt4_8k
            Image Resolution: dalle_512
            Total monthly cost: $0.7
          

            Prompt Model: gpt4_8k
            Image Resolution: dalle_1024
            Total monthly cost: $0.76
          

            Prompt Model: gpt4_32k
            Image Resolution: dallee_256
            Total monthly cost: $0.8
          

            Prompt Model: gpt4_32k
            Image Resolution: dalle_512
            Total monthly cost: $0.86
          

            Prompt Model: gpt4_32k
            Image Resolution: dalle_1024
            Total monthly cost: $0.92
          

            Prompt Model: gpt35t_4k
            Image Resolution: dallee_256
            Total monthly cost: $0.49
          

            Prompt Model: gpt35t_4k
            Image Resolution: dalle_512
            Total monthly cost: $0.55
         